# TODO: Yes you should understand backprop
Blog: https://karpathy.medium.com/yes-you-should-understand-backprop-e2f06eab496b

1. Understand the starter code.
2. Write the backward pass manually for:
   1. Whole Linear + Non-linear + BN + softmax.
   2. Faster softmax. (understand the derivation)
   3. Faster BN. (understand the derivation)
   4. All together.
3. Back to starter code and try to understand what intermediate variables are needed.

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [ ]:
# read in all the words
words = open('../names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

In [ ]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

In [ ]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

In [ ]:
# ok biolerplate done, now we get to the action:

In [ ]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [ ]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

In [ ]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
print(Xb.shape)
print(Yb.shape)

In [ ]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability of exp()
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

# Exercise 1
1. First identify the shape, careful with the broadcasting or reshaping.
2. Then try to think about a simpler/smaller example, such as -1/3 * (a + b + c) for the last computation (logprobs -> loss).
3. Then try to calculate the gradient using the chain rule.
4. A node in the chain rule, and a term in the matrix, can be treated the same way.

In [ ]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

# Cross entropy
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = - 1./n # mean.
dprobs = 1. / probs * dlogprobs # log_e. (It can be observed that 1./probs boost those with low prob)
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)   # linear func * derivative of broadcasting (if a node is used multiple times, its gradient just sums. Or matrix derivative can be use)
dcounts = counts_sum_inv * dprobs # first contribution of counts.
dcounts_sum = -1./(counts_sum**2) * dcounts_sum_inv # inverse.
dcounts += 1.0 * dcounts_sum.repeat(1, counts.shape[1])   # Distribute the aggregation.
dnorm_logits = norm_logits.exp() * dcounts  # exp()
dlogit_maxes = -1 * dnorm_logits.sum(1, keepdim=True) # Broadcasting: node used multiple times just sum up!!!! This should be zero, because normalization won't affect the probs later.
dlogits = 1. * dnorm_logits.clone()
dlogits[range(n), logits.argmax(dim=1)] += 1. * dlogit_maxes.squeeze(1) # Chain rule & Distribute the max.
# Linear
dh = dlogits @ W2.T # Matrix derivative.
dW2 = h.T @ dlogits
db2 = (1. * dlogits).sum(dim=0) # Broadcasting.
dhpreact = (1 - torch.tanh(hpreact)**2) * dh # Tanh
# BN
dbngain = (bnraw * dhpreact).sum(dim=0) # Forward: broadcasting fisrt then multiply. Backward: multiply first then broadcasting.
dbnbias = (1. * dhpreact).sum(dim=0)
dbnraw = bngain * dhpreact
dbnvar_inv = (bndiff * dbnraw).sum(dim=0)
dbnvar = (-1/2. * (bnvar + 1e-5) ** (-3/2)) * dbnvar_inv
dbndiff2 = (1 / (n - 1) * dbnvar).repeat(bndiff2.shape[0], 1) # Distribute the aggregation.
dbndiff = 2. * bndiff * dbndiff2
dbndiff += bnvar_inv * dbnraw
dbnmeani = -1. * dbndiff.sum(dim=0)
dhprebn = 1. * dbndiff
dhprebn += (1 / n * dbnmeani).repeat(hprebn.shape[0], 1)
# Embed/Linear1
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = (1. * dhprebn).sum(dim=0)
demb = dembcat.view(emb.shape) # View back
dC = torch.zeros_like(C) # This is tricky. Backward the indexing.
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k, j]
        dC[ix] += demb[k, j]

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

# Exercise 2

In [ ]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

In [ ]:
# backward pass

dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits /= n

cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9

In [ ]:
logits.shape, Yb.shape

In [ ]:
F.softmax(logits, 1)[0]

In [ ]:
dlogits[0] * n

In [ ]:
dlogits[0].sum()

In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(dlogits.detach(), cmap='gray')

# Exercise 3

In [ ]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

In [ ]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))

cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

In [ ]:
dhprebn.shape, bngain.shape, bnvar_inv.shape, dbnraw.shape, dbnraw.sum(0).shape

# Exercise 4

In [40]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

n_embed = 10
n_hidden = 200
block_size = 3
vocab_size = 27

g = torch.Generator().manual_seed(2147483647) # for reproducibility
embed = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((block_size * n_embd, n_hidden), generator=g) * (5/3) * ((block_size * n_embd)**(-1/2))
b1 = torch.randn(n_hidden, generator=g) * 0.1
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1
bngain = torch.randn((1, n_hidden), generator=g) * 0.1 + 1.0
bnbias = torch.randn((1, n_hidden), generator=g) * 0.1

parameters = [embed, W1, b1, W2, b2, bngain, bnbias]
for p in parameters:
    p.require_grads = True

max_steps = 200000
batch_size = 32

with torch.no_grad():
    for epoch in range(max_steps):
        ix = torch.randint(0, Xtr.shape[0], (batch_size, ), generator=g)
        X = Xtr[ix]
        Y = Ytr[ix]

        # Forward
        X = embed[X]
        X = X.view(batch_size, -1)
        hprebn = X @ W1 + b1
        
        bnmean = hprebn.mean(0, keepdim=True)
        bnvar = hprebn.var(0, keepdim=True, unbiased=True)
        bnvar_inv = (bnvar + 1e-5)**-0.5
        bnraw = (hprebn - bnmean) * bnvar_inv
        hpreact = bngain * bnraw + bnbias

        h = torch.tanh(hpreact)
        logits = h @ W2 + b2
        loss = F.cross_entropy(logits, Y)

        # Backward
        for p in parameters:
            p.grad = None

        # Cross entropy
        dlogits = F.softmax(logits, dim=1)
        dlogits[range(batch_size), Y] -= 1.
        dlogits /= n
        # Linear
        dh = dlogits @ W2.T # Matrix derivative.
        dW2 = h.T @ dlogits
        db2 = (1. * dlogits).sum(dim=0) # Broadcasting.
        dhpreact = (1 - torch.tanh(hpreact)**2) * dh # Tanh
        # BN
        dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
        dbnbias = dhpreact.sum(0, keepdim=True)
        dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
        # Embed/Linear1
        dembcat = dhprebn @ W1.T
        dW1 = embcat.T @ dhprebn
        db1 = (1. * dhprebn).sum(dim=0)
        demb = dembcat.view(emb.shape) # View back
        dC = torch.zeros_like(C) # This is tricky. Backward the indexing.
        for k in range(Xb.shape[0]):
            for j in range(Xb.shape[1]):
                ix = Xb[k, j]
                dC[ix] += demb[k, j]
        grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]

        lr = 0.1 if epoch < 10000 else 0.01
        for p, grad in zip(parameters, grads):
            p += -lr * grad

        if epoch % 10000 == 0: # print every once in a while
            print(f'{epoch:7d}/{max_steps:7d}: {loss.item():.4f}')

      0/ 200000: 3.9508
  10000/ 200000: 2.5499
  20000/ 200000: 2.5571
  30000/ 200000: 2.5101
  40000/ 200000: 2.5027
  50000/ 200000: 2.2282
  60000/ 200000: 2.4268
  70000/ 200000: 2.2702
  80000/ 200000: 2.1272
  90000/ 200000: 2.5008
 100000/ 200000: 2.4313
 110000/ 200000: 2.5056
 120000/ 200000: 2.6770
 130000/ 200000: 2.6847
 140000/ 200000: 2.4810
 150000/ 200000: 2.4804
 160000/ 200000: 2.3089
 170000/ 200000: 2.2274
 180000/ 200000: 2.2925
 190000/ 200000: 2.3527


In [ ]:
# useful for checking your gradients
# for p,g in zip(parameters, grads):
#   cmp(str(tuple(p.shape)), g, p)

TypeError: all() received an invalid combination of arguments - got (bool), but expected one of:
 * (Tensor input, *, Tensor out = None)
 * (Tensor input, tuple of ints dim = None, bool keepdim = False, *, Tensor out = None)
 * (Tensor input, int dim, bool keepdim = False, *, Tensor out = None)
 * (Tensor input, name dim, bool keepdim = False, *, Tensor out = None)


In [ ]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)


In [41]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.388674736022949
val 2.3984973430633545


In [ ]:
# I achieved:
# train 2.0718822479248047
# val 2.1162495613098145

In [42]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # ------------
      # forward pass:
      # Embedding
      emb = C[torch.tensor([context])] # (1,block_size,d)      
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # ------------
      # Sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

eria.
imyazleeelend.
ryale.
emmruiengrlen.
adee.
doeliianni.
jenleieia.
ananarlelyzioh.
kay.
nysadbergshianest.
jaie.
jelson.
bruoriu.
zeneda.
yyleli.
els.
kay.
avennyah.
hal.
aadyansui.
